<a href="https://colab.research.google.com/github/francji1/01NAEX/blob/main/HW/2022/01NAEX_HW01_pulse_1_solution_teamA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#01NAEX Home project 01

*Team:*
* Frátrik Emanuel
* Kysilka Lukáš
* Nguyenová Hana
* Ružejnikov Jura
* Stojaspal Pavel

In [ ]:
knitr::opts_chunk$set(echo = TRUE)
# get requirements for this project
list_of_packages <- c("tidyverse", "lattice", "pwr", "MASS","GGally","agridat","agricolae")
missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
if(length(missing_packages)) install.packages(missing_packages)
lapply(list_of_packages, library, character.only = TRUE)

######################
# check your settings
#getwd()
#print(.libPaths())
#print(sessionInfo())

In [ ]:
print(version)

# Research task:

The goal of this research task is to find out the effect of exercise on resting heart rate.



# Descrition:

You have the same number of operators as team members. Each of you will run the experiment, i.e. physical exercises and resting heart rate measurement.
Choose 3 differently difficult exercises, (for example squats, push-ups, sit-ups, chair climbs, plank, mathematical computation, ...) and "iteration" whose levels are 10, 20, 30 (number of repetitions, number of seconds for plank, ...) affect the increase in heart rate per minute. Measure your heart rate at rest (you should have it at a similar level before each subsequent measurement) and then immediately after the exercise. Simply measure for 15 seconds and multiply the measurement by 4. In total, take 9 measurements each.

# Submission: 

Submit the elaborated measurement by October 19, 2022 in pdf + Rmd/jpnb format.

# Questions: 

 1. Measure the data and set up the experiment so that it is *randomized*. Be sure to record the order of the measurements and the operator (member of the group taking the measurements) for each measurement. The number of replications for operator, exercise and iteration is one.

In [ ]:
#data <- read.csv("data.csv")
data <- read.csv("https://raw.githubusercontent.com/francji1/01NAEX/main/HW/2022/NAEX_2022_HW01_sol_teamA.csv")

In [ ]:
data <- data %>% transmute(operator = as.factor(operator),
                           cvik = as.factor(cvik),
                           iterace = as.factor(iterace),
                           tep = tep)
data

done



2. Calculate the basic statistics (mean, median and sd for each variable). Display the measured data (box plot, interaction plot, effects plot) and comment what we can predict about the outcome from the given figures before the analysis.

In [ ]:
summary(data)
str(data)

In [ ]:
data %>%                               # Summary by group using dplyr
  group_by(operator) %>% 
  summarize(min = min(tep),
            q1 = quantile(tep, 0.25),
            median = median(tep),
            mean = mean(tep),
            q3 = quantile(tep, 0.75),
            max = max(tep),
            std = sd(tep))
data %>%                               # Summary by group using dplyr
  group_by(cvik) %>% 
  summarize(min = min(tep),
            q1 = quantile(tep, 0.25),
            median = median(tep),
            mean = mean(tep),
            q3 = quantile(tep, 0.75),
            max = max(tep),
            std = sd(tep))

data %>%                               # Summary by group using dplyr
  group_by(iterace) %>% 
  summarize(min = min(tep),
            q1 = quantile(tep, 0.25),
            median = median(tep),
            mean = mean(tep),
            q3 = quantile(tep, 0.75),
            max = max(tep),
            std = sd(tep))



In [ ]:
library(ggplot2)

# **Box plots**

In [ ]:
data %>%  
          ggplot(aes(x=operator, y=tep, fill=operator)) + 
          geom_boxplot() + 
          stat_summary(fun="mean", size=1, color='purple') + 
          geom_jitter(color="black", size=0.9, alpha=0.5) +
          theme(legend.position="none")
          #geom_point(size=3, shape=16, color='red')

Na základe boxplotov sa dá usúdiť, že medzi jednotlivými blokmi/ operátormi nie je výrazný rozdiel v priemernom tepe
naprieč všetkými cvikmi a ich intenzitou (počtom iterácii). Variance v rámci blokov sa taktiež zdá byť rozumne rovnaká.

In [ ]:
data %>% ggplot(aes(x=cvik, y=tep, fill=cvik)) + 
          geom_boxplot() + 
          stat_summary(fun="mean", size=1, color='purple') +
          geom_jitter(color="black", size=0.9, alpha=0.5) +
          theme(legend.position="none")

Na druhú stranu boxplot vyššie naznačuje, že medzi cvikmi drep a židle nie je významný rozdiel v priemernom tepe ale medzi plankom a židli je.

In [ ]:
data %>% ggplot(aes(x=iterace, y=tep, fill=iterace)) + 
          geom_boxplot() + 
          stat_summary(fun="mean", size=1, color='purple') +
          geom_jitter(color="black", size=0.9, alpha=0.5) +
          theme(legend.position="none")

Podobne môžme očakávať, že intenzita cvičenia meraná počtom iterácii cviku má vplyv na priemerný tep. Tento predpoklad potvrdzuje aj boxplot vyššie ale len do istej miery nakoľko medzi interaciami s 20 a 30 opakovaniami sa nezdá byť výrazný rozdiel čo môže byť sposobené akousi saturáciou tepovej intenzity keďže srdce má tiež svoje kapacity...

## **Interaction plots**

In [ ]:
data %>% 
  group_by(operator, cvik) %>% 
  summarise(mean_tep = mean(tep)) %>%
  ggplot() +
  aes(x = cvik, y = mean_tep, color = operator) +
  geom_line(aes(group = operator)) +
  geom_point() +
  labs(title="Interaction plot [cvik|operator]", size=10)

In [ ]:
data %>% 
  group_by(iterace, operator) %>% 
  summarise(mean_tep = mean(tep)) %>%
  ggplot() +
  aes(x = iterace, y = mean_tep, color = operator) +
  geom_line(aes(group = operator)) +
  geom_point() +
  labs(title="Interaction plot [iterace|operator]", size=10)

V blokovom dizajne experimentu sa typicky nepredpokladá interackcia medzi blokovou premennou (operator v našom prípade) a vysvetľujúcimi. Dva obrázky vyššie poukazujú na to, že v rámci blokov sú trendy voči vývoju priemerného tepu podobné. Iterácia a operátor zjavne nevykazujú interakciu ale medzi operátorom a typom cviku môže existovať malá interakcia čo sa dá predpokladať vzhľadom k rôznej fyzickej kondicii jednotlivých operátorov.

In [ ]:
data %>% 
  group_by(cvik, iterace) %>% 
  summarise(mean_tep = mean(tep)) %>%
  ggplot() +
  aes(x = cvik, y = mean_tep, color = iterace) +
  geom_line(aes(group = iterace)) +
  geom_point() +
  labs(title="Interaction plot [cvik|iterace]", size=10)

In [ ]:
data %>% 
  group_by(cvik, iterace) %>% 
  summarise(mean_tep = mean(tep)) %>%
  ggplot() +
  aes(x = iterace, y = mean_tep, color = cvik) +
  geom_line(aes(group = cvik)) +
  geom_point() +
  labs(title="Interaction plot [iterace|cvik]", size=10)

Interakčný plot medzi iteráciou a cvikom naznačuje, že medzi nimi môže dochádzať k nejakej interakcii. Zvláštne je, že v prípade drepov dochádza pri 30-násobnom opakovaní k zníženiu priemernej tepovej frekvencie oproti 20-násobnému opakovaniu.

# **Effect plots**

In [ ]:
data %>% 
  group_by(operator) %>% 
  summarise(mean_tep = mean(tep)) %>%
  ggplot(aes(x = operator, y = mean_tep, group=1, color=operator)) +
  geom_line() +
  geom_point() +
  labs(title="Effect plot", size=10)

In [ ]:
data %>% 
  group_by(cvik) %>% 
  summarise(mean_tep = mean(tep)) %>%
  ggplot(aes(x = cvik, y = mean_tep, group=1, color=cvik)) +
  geom_line() +
  geom_point() +
  labs(title="Effect plot", size=10)

In [ ]:
data %>% 
  group_by(iterace) %>% 
  summarise(mean_tep = mean(tep)) %>%
  ggplot(aes(x = iterace, y = mean_tep, group=1, color=iterace)) +
  geom_line() +
  geom_point() +
  labs(title="Effect plot", size=10)

Z effect plotov plynú rovnaké závery ako z boxplotov
takže v nich nevidíme veľký zmysel...

 
 
3.  Focus especially on the factors exercise and iteration, operator is a nuisance parameter. Run one-way analysis of variance for each factor separately. Are there significance differences in means of hart rate increases? For each factor: perform Tukey HSD and Fisher LSD test for pairwise comparison of means of individual groups.

### Cvik

In [ ]:
# Compute the analysis of variance
res_aov_cvik <- aov(tep ~ cvik, data = data)
# Summary of the analysis
summary(res_aov_cvik)

In [ ]:
Tukey_cvik <- TukeyHSD(res_aov_cvik ,which="cvik")
Tukey_cvik
plot(Tukey_cvik) 

In [ ]:
FisherLSD_cvik <- LSD.test(res_aov_cvik, "cvik")
plot(FisherLSD_cvik)

In [ ]:
bartlett.test(tep ~ cvik, data=data)

In [ ]:
plot(lm(tep ~ cvik, data=data))

Cviky dřep a židle mají statisticky stejné střední hodnoty. Cvik plank má odlišnou střední hodnotu od těchto skupin.

### Operator

In [ ]:
# Compute the analysis of variance
res_aov_operator <- aov(tep ~ operator, data = data)
# Summary of the analysis
summary(res_aov_operator)

In [ ]:
bartlett.test(tep ~ operator, data=data)

Anova nenaznačuje signifikantný rozdiel medzi jednotlivými operátormi.

### Iterace

In [ ]:
# Compute the analysis of variance
res_aov_iterace <- aov(tep ~ iterace, data = data)
# Summary of the analysis
summary(res_aov_iterace)

In [ ]:
Tukey_iterace <- TukeyHSD(res_aov_iterace ,which="iterace")
Tukey_iterace
plot(Tukey_iterace) 

In [ ]:
FisherLSD_iterace <- LSD.test(res_aov_iterace, "iterace")
plot(FisherLSD_iterace)

In [ ]:
bartlett.test(tep ~ iterace, data=data)

In [ ]:
plot(lm(tep ~ iterace, data=data))

4.  Analyse the data using ANOVA (take all variables as a factor).
       * With the assumption that each of the participants in the experiment forms a block.
       * Discuss results, verify assumptions, plot QQ-plot, residuals x fitted values, residuals x factor levels.
       * Discuss the effect of the experimental design and the evolution of the residuals over time (number of measurements). 
       * If the measured data do not meet the assumptions for using ANOVA, discuss possible transformations (logarithmic, Box-Cox, ...)  and the reason why the data do not meet the assumptions? 


In [ ]:
summary(aov(tep ~ operator + cvik + iterace, data=data))
plot(aov(tep ~ operator + cvik + iterace, data=data))
plot(aov(tep ~ cvik + iterace + operator, data=data),5)
plot(aov(tep ~ iterace + operator + cvik, data=data),5)

Můžeme vidět, že proměnná operátor je významná. Z grafů můžeme vidět, že jedno pozorování č. 8 je odlehlé a ovlivňuje křivky. Normalita z grafu je v pořádku. 

In [ ]:
plot(aov(tep ~ operator + cvik + iterace, data=data),4)

Díky náhodnému provedení jednotlivých cviků nejsou rezidua  tolik závislá na čase. Každý cvik obsahuje rezidua z různých časových okamžiků, díky tomu je eliminována únava operátora.
Díky uspořádání experimentu rezidua odpovídají normálnímu rozdělení, což lze vypozorovat z výše uvedených grafů. Tento závěr potvrzuje i Shapiro-Wilkův test uvedený níže.

In [ ]:
# Extract the residuals
aov_residuals <- residuals(object = aov(tep ~ operator + cvik + iterace, data=data) )
# Run Shapiro-Wilk test
shapiro.test(x = aov_residuals )

In [ ]:
summary(aov(log(tep) ~  cvik + iterace + operator, data=data))
plot(aov(log(tep) ~ operator + cvik + iterace, data=data))

In [ ]:
# anova bez operator
summary(aov(tep ~ cvik + iterace, data=data))

      
5. Choose one level of iteration factor and *even if it does not meet the assumptions for using ANOVA*, solve the following: 
   * Calculate the power of the test in the ANOVA for 3 levels of exercises and **n** operators.
   * Assume the standard deviation of the disturbances to be 4 for the experiment conducted
     and the maximum (for us significant) difference we want to detect is 5 bpm.
     Calculate the number of operators needed for the power of the resulting test to be greater than 0.9.


In [ ]:
b = seq(2,27,by=1)
a = 3 
D = 5
sigma = 4
powers <- power.anova.test(groups=a,n=b,between.var = (D^2/2)/(a),
within.var=sigma^2,sig.level=.05)$power
rbind(b , powers)

Pro operátor 26, je síla testu vyšší než 0.9.


6. Create a regression model where you do not consider blocks and the iteration of exercises is the quantitative variable.
      * Try to add a squared variable to the model and compare the two regression models with each other and select the more appropriate one.
      * Check the assumptions for using the chosen model.


In [ ]:
data1 <- data %>% transmute(operator = as.factor(operator),
                           cvik = as.factor(cvik),
                           iterace = as.numeric(iterace),
                           tep = tep)

In [ ]:
library(MASS)
# Fit the full model 
full1.model <- lm(tep ~ cvik + iterace + cvik:iterace, data = data1)
# Stepwise regression model
step1.model <- stepAIC(full1.model, direction = "both", 
                      trace = FALSE)
summary(step1.model)

In [ ]:
# Fit the full model 
full2.model <- lm(tep ~ cvik + I(iterace^2) + iterace + cvik:iterace, data = data1)
# Stepwise regression model
step2.model <- stepAIC(full2.model, direction = "both", trace = FALSE)
summary(step2.model)

In [ ]:
plot(step1.model)

In [ ]:
anova(full1.model,full2.model)

In [ ]:
anova(step1.model,step2.model)

In [ ]:
bartlett.test(tep ~ iterace, data=data1)
bartlett.test(tep ~ cvik, data=data1)

In [ ]:
plot(step1.model)

Anova test neukázal, že mezi modely jsou významné rozdíly.


7. If you based your previous answers on the p-value, discuss following.  Does p-value provide a good measure of evidence regarding a model or hypothesis?

In [ ]:
model1 <- lm(tep ~ cvik + iterace + cvik:iterace + operator, data = data1)
model2 <- lm(tep ~ operator + cvik + I(iterace^2) + iterace + cvik:iterace, data = data1)
anova(model1, model2)

Nakoľko anova poukazuje na nesignifikantný rozdiel medzi modelmi tak je lepšie uvažovať model jednoduchší. Na druhú stranu v modeloch nezohľadňujeme známy a významný (ako bolo ukázané v 4.) efekt blokov čo môže viesť k zavádzajúcim záverom ohľadom porovanávania dvoch vnorených modelov. (Po zohľadnení efektu blokov sme dospeli k rovnakému záveru a síce na hladine 5% nie je medzi modelmi významný rozdiel a preto je lepšie pracovať s jednoduchším...)

Note: In case the blocks are highly dominant and none of the other variables appear significant,
      use only the "operator with the best data for academic purposes" for the resulting analysis.